In [6]:
#https://figshare.com/articles/smile_annotations_final_csv/3187909/2

In [7]:
#Data Analysis and Preprocessing

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [9]:
#selected from
#https://github.com/PyThaiNLP/wisesight-sentiment/blob/master/kaggle-competition/competition.ipynb

In [2]:
import numpy as np
from pythainlp import word_tokenize
import re
import emoji

In [3]:
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def process_text(text):
    #pre rules
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)
    
    #tokenize
    res = [word for word in word_tokenize(res) if word and not re.search(pattern=r"\s+", string=word)]
    
    #post rules
    res = ungroup_emoji(res)
    
    return res

In [26]:
all_df = pd.read_csv('Data2/all_df.csv')
test_df = pd.read_csv('Data2/test_df.csv')

all_df['processed'] = all_df.texts.map(lambda x: '|'.join(process_text(x)))
all_df['wc'] = all_df.processed.map(lambda x: len(x.split('|')))
all_df['uwc'] = all_df.processed.map(lambda x: len(set(x.split('|'))))

test_df['processed'] = test_df.texts.map(lambda x: '|'.join(process_text(x)))
test_df['wc'] = test_df.processed.map(lambda x: len(x.split('|')))
test_df['uwc'] = test_df.processed.map(lambda x: len(set(x.split('|'))))

In [27]:
print(all_df.shape)
print(test_df.shape)

(24063, 5)
(2674, 5)


In [28]:
all_df.category.value_counts()

neu    13105
neg     6140
pos     4300
q        518
Name: category, dtype: int64

In [32]:
#for faster training due to gpu limitations
r = 0.15
neucat = all_df[all_df.category=='neu']
negcat = all_df[all_df.category=='neg']
poscat = all_df[all_df.category=='pos']
qcat = all_df[all_df.category=='q']

neucat = neucat[0:int(len(neucat)*0.15)]
negcat = negcat[0:int(len(negcat)*0.15)]
poscat = poscat[0:int(len(poscat)*0.15)]
qcat = qcat[0:int(len(qcat)*0.15)]

In [33]:
df = neucat.append([negcat,poscat,qcat])

In [34]:
df.category.value_counts()

neu    1965
neg     921
pos     645
q        77
Name: category, dtype: int64

In [35]:
possible_labels = df.category.unique()

In [36]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [37]:
label_dict

{'neu': 0, 'neg': 1, 'pos': 2, 'q': 3}

In [38]:
df['label'] = df.category.replace(label_dict)
df.head()

,category,texts,processed,wc,uwc,label
0,neu,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,ประเทศ|เรา|ผลิต|และ|ส่งออก|ยาสูบ|เยอะ|สุด|ใน|โ...,13,13,0
1,neu,คะ,คะ,1,1,0
3,neu,😅😅😅,😅|xxrep,2,2,0
4,neu,สวัสดีวันพุธ แนน อะไรนะ,สวัสดี|วัน|พุธ|แนน|อะไร|นะ,6,6,0
5,neu,ก้อนขอบพระคุณมากๆนะคร้าบ ที่มาหาก้อนและชมเชยกา...,ก้อน|ขอบพระคุณ|มาก|ๆ|นะ|คร้าบ|ที่|มาหา|ก้อน|แล...,20,17,0


In [39]:
#Train Test Split

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
#X_train, X_val created to be able to access in dataframe via df.loc[]
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values, #X
    df.label.values, #y
    test_size=0.15,
    random_state=17,
    stratify=df.label.values #ensure all categories are trained since there is uneven distribution
)

In [42]:
X_val.shape

(542,)

In [43]:
df['data_type'] = ['not_set']*df.shape[0]

In [44]:
df.head()

,category,texts,processed,wc,uwc,label,data_type
0,neu,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,ประเทศ|เรา|ผลิต|และ|ส่งออก|ยาสูบ|เยอะ|สุด|ใน|โ...,13,13,0,not_set
1,neu,คะ,คะ,1,1,0,not_set
3,neu,😅😅😅,😅|xxrep,2,2,0,not_set
4,neu,สวัสดีวันพุธ แนน อะไรนะ,สวัสดี|วัน|พุธ|แนน|อะไร|นะ,6,6,0,not_set
5,neu,ก้อนขอบพระคุณมากๆนะคร้าบ ที่มาหาก้อนและชมเชยกา...,ก้อน|ขอบพระคุณ|มาก|ๆ|นะ|คร้าบ|ที่|มาหา|ก้อน|แล...,20,17,0,not_set


In [45]:
df.loc[X_train,'data_type'] = 'train'
df.loc[X_val,'data_type'] = 'val'

In [46]:
df.groupby(['category','label','data_type']).count()

texts  processed    wc   uwc
category label data_type                              
neg      1     train        783        783   783   783
               val          138        138   138   138
neu      0     train       1670       1670  1670  1670
               val          295        295   295   295
pos      2     train        548        548   548   548
               val           97         97    97    97
q        3     train         65         65    65    65
               val           12         12    12    12

In [20]:
#Bert Tokenizer

In [47]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [48]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-multilingual-cased',  #use pretrained bert tokenizer
    #do_lower_case=True  #turn sentences into lower case
)

In [49]:
#Multilingual automatically detects language
#Tokenizer tests

tokenizer.tokenize('อะไร')

tokenizer.tokenize('สวัสดีปีใหม่ค่ะ')

tokenizer.tokenize('hi this is Joe')

In [50]:
#tokenizer.get_vocab() #gets a lot of languages

In [51]:
encoded_data_train = tokenizer.batch_encode_plus( #iterates over sentences
    df[df.data_type=='train'].texts.values,  #get the sentences for training
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(  #iterates over sentences
    df[df.data_type=='val'].texts.values,  #get the sentences for training
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)
#why 256?? checked with the Project.ipynb has max word counts = 103

#inputs for BERT
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [52]:
dataset_train = TensorDataset(input_ids_train,
                             attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val,
                             attention_masks_val, labels_val)

In [53]:
len(dataset_train)

3066

In [54]:
len(dataset_val)

542

In [55]:
#Set up BERT pretrained

In [1]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels = len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

In [30]:
#Create data loaders helps iterate through dataset batches nicely

In [58]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [59]:
batch_size = 4 #32 #not to overload memory

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

In [60]:
#Setting Up Optimizer

In [61]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [62]:
#find weights
optimizer = AdamW(
    model.parameters(),
    lr=1e-5, #learning rate should be approx. 2e-5 > 5e-5
    eps=1e-8 #epsilon
)

In [63]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs #total training steps = data to train*epochs
)

In [64]:
#Define performance metrics

In [65]:
import numpy as np

In [66]:
from sklearn.metrics import f1_score

In [67]:
def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,preds_flat,average='weighted')

In [68]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class:{label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [69]:
#create training loop

In [70]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


In [71]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #cuda if gpu is available
model.to(device)

print(device)

cpu


In [72]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs) #**inputs flattens dictionary into inputs
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()  #incase want to use gpu??
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1,epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    
    #to show progress by tqdm
    progress_bar = tqdm(dataloader_train,
                       desc = 'Epoch {:1d}'.format(epoch),
                       leave=False, #will replace tqdm when done
                        disable=False
                       )
    
    for batch in progress_bar:
        
        model.zero_grad() #start with 0 gradient
        
        batch = tuple(b.to(device) for b in batch)
        
        #inputs into model
        inputs = {
            'input_ids':batch[0],
            'attention_mask':batch[1],
            'labels':batch[2]
        }
        
        outputs = model(**inputs)
        
        loss=outputs[0]
        loss_train_total += loss.item()
        loss.backward() #backward propagation??
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        #help with the calculating gradient by clipping
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'Models/ThaiBERT_ft_epoch{epoch}.model')
    
    #displays
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg} ')
    
    #for validation set
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions,true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

In [ ]:
#Loading and Evaluating Model

In [75]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [76]:
model.to(device) #select the device
pass

In [77]:
model.load_state_dict(
    torch.load('Models/ThaiBERT_ft_epoch10.model',
              map_location=torch.device('cpu')))

<All keys matched successfully>

In [78]:
_,predictions,true_vals = evaluate(dataloader_val)

In [79]:
accuracy_per_class(predictions, true_vals)

Class:neu
Accuracy: 233/295

Class:neg
Accuracy: 89/138

Class:pos
Accuracy: 41/97

Class:q
Accuracy: 2/12



In [ ]:
#Google Colab --GPU Instance
#batch_size = 32
#epoch =10

In [ ]:
233/295